# Netflix Title Analysis

This notebook uploads the `netflix_titles.csv` dataset, performs cleaning, visualizes key insights, and stores the cleaned data in an SQLite database for additional querying.

In [ ]:
import io
import sqlite3
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from google.colab import files

sns.set_theme(style="whitegrid")
plt.rcParams.update({"figure.figsize": (10, 6)})

## Upload the dataset

Run the following cell to upload `netflix_titles.csv` using the Colab file picker.

In [ ]:
uploaded = files.upload()

# Expect a key named 'netflix_titles.csv'
file_name = next(iter(uploaded))
raw_bytes = uploaded[file_name]

## Load and inspect the data

In [ ]:
df = pd.read_csv(io.BytesIO(raw_bytes))

print(f"Loaded {len(df):,} rows.")
df.head()

## Clean the dataset

Steps:
- Fill missing `director`, `cast`, and `country` values with "Unknown".
- Drop rows without a `title`.
- Convert `date_added` to datetime.
- Add `year_added` and `month_added` columns.
- Drop duplicate `show_id` entries (keep the first occurrence).

In [ ]:
cleaned = df.copy()

# Fill missing director, cast, and country values
cleaned[['director', 'cast', 'country']] = cleaned[['director', 'cast', 'country']].fillna('Unknown')

# Drop rows with missing titles
cleaned = cleaned.dropna(subset=['title'])

# Convert date_added to datetime
cleaned['date_added'] = pd.to_datetime(cleaned['date_added'], errors='coerce')

# Add year and month columns
cleaned['year_added'] = cleaned['date_added'].dt.year
cleaned['month_added'] = cleaned['date_added'].dt.month

# Drop duplicate show_id rows
cleaned = cleaned.drop_duplicates(subset='show_id', keep='first')

cleaned.info()

## Movies vs TV Shows

In [ ]:
type_counts = cleaned['type'].value_counts().reset_index()
type_counts.columns = ['type', 'count']

ax = sns.barplot(data=type_counts, x='type', y='count', palette='pastel')
ax.bar_label(ax.containers[0])
plt.title('Distribution of Content Types')
plt.xlabel('Type')
plt.ylabel('Count')
plt.show()

## Top 10 countries by content count

In [ ]:
country_counts = (
    cleaned[cleaned['country'] != 'Unknown']['country']
    .str.split(',')
    .explode()
    .str.strip()
    .value_counts()
    .head(10)
    .reset_index()
)
country_counts.columns = ['country', 'count']

ax = sns.barplot(data=country_counts, y='country', x='count', palette='mako')
ax.bar_label(ax.containers[0])
plt.title('Top 10 Countries by Content Count')
plt.xlabel('Count')
plt.ylabel('Country')
plt.show()

## Top 10 genres from `listed_in`

In [ ]:
genre_counts = (
    cleaned['listed_in']
    .fillna('Unknown')
    .str.split(',')
    .explode()
    .str.strip()
    .value_counts()
    .head(10)
    .reset_index()
)
genre_counts.columns = ['genre', 'count']

ax = sns.barplot(data=genre_counts, y='genre', x='count', palette='rocket')
ax.bar_label(ax.containers[0])
plt.title('Top 10 Genres')
plt.xlabel('Count')
plt.ylabel('Genre')
plt.show()

## Content added per year

In [ ]:
yearly_additions = (
    cleaned.dropna(subset=['year_added'])
    .groupby('year_added')['show_id']
    .count()
    .reset_index()
    .sort_values('year_added')
)
yearly_additions.columns = ['year_added', 'count']

ax = sns.lineplot(data=yearly_additions, x='year_added', y='count', marker='o')
plt.title('Content Added Per Year')
plt.xlabel('Year Added')
plt.ylabel('Number of Titles')
plt.xticks(rotation=45)
plt.show()

## Ratings distribution

In [ ]:
rating_counts = cleaned['rating'].value_counts(dropna=False).reset_index()
rating_counts.columns = ['rating', 'count']

ax = sns.barplot(data=rating_counts, y='rating', x='count', palette='crest')
ax.bar_label(ax.containers[0])
plt.title('Ratings Distribution')
plt.xlabel('Count')
plt.ylabel('Rating')
plt.show()

## Persist cleaned data to SQLite

In [ ]:
conn = sqlite3.connect('netflix.db')
cleaned.to_sql('titles', conn, if_exists='replace', index=False)

print('Rows inserted:', cleaned.shape[0])

## Sample SQL queries

In [ ]:
queries = {
    'total_titles': '''
        SELECT COUNT(*) AS total_titles FROM titles;
    ''',
    'movies_by_year': '''
        SELECT release_year, COUNT(*) AS movie_count
        FROM titles
        WHERE type = 'Movie'
        GROUP BY release_year
        ORDER BY release_year DESC
        LIMIT 10;
    ''',
    'top_directors': '''
        SELECT director, COUNT(*) AS title_count
        FROM titles
        WHERE director != 'Unknown'
        GROUP BY director
        ORDER BY title_count DESC
        LIMIT 10;
    ''',
}

cursor = conn.cursor()
results = {}
for name, query in queries.items():
    cursor.execute(query)
    results[name] = cursor.fetchall()

results